In [1]:
# builtins
import locale
import math
import glob
import pathlib
import functools
import logging
import time
import datetime

# numerical stuff
import pickle5 as pickle
import tables


In [2]:
# monkey patch the tables library to support pickle5, files were created using Python 3.8
tables.atom.pickle = pickle
tables.atom.pickle

<module 'pickle5' from '/home/guusvh/venvs/main/lib/python3.6/site-packages/pickle5/__init__.py'>

In [3]:
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Conv2DTranspose, Reshape
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, Input, UpSampling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.python import ipu

import libpvti as pvti

# plotting
import matplotlib.pyplot as plt

logging.basicConfig(level=logging.INFO)


In [4]:
# Standard IPU TensorFlow setup.
ipu_config = ipu.config.IPUConfig()
ipu_config.auto_select_ipus = 2
ipu_config.configure_ipu_system()

# Create an execution strategy.
strategy = ipu.ipu_strategy.IPUStrategy()

# Configure profiling
pvti.getCurrentTraceFilename()
#!export PVTI_OPTIONS='{"enable":"true"}'

'Fri_Oct_29_06:10:05_2021_EDT_160706.pvti'

In [5]:
data_path_train = 'gs://bathy_sample/processed/20211013/train_data_mask_no_schematic'
data_path_test = 'gs://bathy_sample/processed/20211013/test_data_mask_no_schematic'
meta_data_path = '/mnt/poddata/data/bathy-emodnet-a-runs.h5'
#all_checkpoints_path = 'gs://bathy_sample/dnn/checkpoints'
all_checkpoints_path = '/mnt/poddata/data/checkpoints'
model_name = 'guus-2d-mlp-cnn-v0.6'
model_path = '/mnt/poddata/data/models'
checkpoints_path = all_checkpoints_path + '/' + model_name

learning_rate = 1e-4
n_epochs = 100
batch_size = 1
steps_per_execution = 32
steps_per_epoch = 896
validation_steps = 320
gradient_accumulation_steps_per_replica = 8
raster_shape = (256, 256, 1)

In [6]:
def tf_parse(eg):
    """parse an example (or batch of examples, not quite sure...)"""

    # here we re-specify our format
    # you can also infer the format from the data using tf.train.Example.FromString
    # but that did not work
    example = tf.io.parse_example(
        eg[tf.newaxis],
        {
            'height': tf.io.FixedLenFeature([], tf.int64),
            'width': tf.io.FixedLenFeature([], tf.int64),
            'depth': tf.io.FixedLenFeature([], tf.int64),
            'bathy': tf.io.FixedLenFeature([], tf.string),
            'hs': tf.io.FixedLenFeature([], tf.string),
            'tm01': tf.io.FixedLenFeature([], tf.string),
            'theta0x': tf.io.FixedLenFeature([], tf.string),
            'theta0y': tf.io.FixedLenFeature([], tf.string),
            'eta': tf.io.FixedLenFeature([], tf.float32),
            'zeta': tf.io.FixedLenFeature([], tf.float32),
            'theta_wavex': tf.io.FixedLenFeature([], tf.float32),
            'theta_wavey': tf.io.FixedLenFeature([], tf.float32),
        },
    )
    bathy = tf.io.parse_tensor(example["bathy"][0], out_type="float32")
    bathy = tf.ensure_shape(bathy, raster_shape)    # ensure shape, to be able to train the model
    hs = tf.io.parse_tensor(example["hs"][0], out_type="float32")
    hs = tf.ensure_shape(hs, raster_shape)
    tm01 = tf.io.parse_tensor(example["tm01"][0], out_type="float32")
    theta0x = tf.io.parse_tensor(example["theta0x"][0], out_type="float32")
    theta0y = tf.io.parse_tensor(example["theta0y"][0], out_type="float32")
    eta = example["eta"]
    zeta = example["zeta"]
    theta_wavex = example["theta_wavex"]
    theta_wavey = example["theta_wavey"]
    attr = tf.stack([eta, zeta, theta_wavex, theta_wavey], axis=1)
    attr = tf.reshape(attr,shape=[-1])
    output = (hs, tm01, theta0x, theta0y)
    output = hs
    return (bathy, attr), output

In [7]:
def get_files(data_path):
    files = tf.io.gfile.glob(data_path + "/" + "*.tfrecords")
    return files

def get_dataset(files):
    """return a tfrecord dataset with all tfrecord files"""
    dataset =  tf.data.TFRecordDataset(files)
    dataset = dataset.map(tf_parse)
    return dataset

In [12]:
def full_model(cnn_input_shape, mlp_input_shape):
    
    mlp_input = Input(mlp_input_shape)
    cnn_input = Input(cnn_input_shape)
    with ipu.keras.PipelineStage(0):
        x = Dense(256, activation='relu')(mlp_input)
        x = Dense(128, activation='relu')(x)
        x = Dense(64, activation='relu')(x)
        x = Dense(32, activation='relu')(x)
        x = Dense(16, activation='relu')(x)
        mlp_output = Dense(4, activation='relu')(x)
    
    
    #with ipu.keras.PipelineStage(1):
        x = Conv2D(16, (3,3), padding="same")(cnn_input)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(2,2))(x)

        x = Conv2D(32, (3,3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(2,2))(x)

        x = Conv2D(64, (3,3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(2,2))(x)

        x = Conv2D(128, (3,3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(2,2))(x)
        
        x = Conv2D(256, (3,3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        
        x = Conv2D(256, (3,3), padding="same")(x)
        x = Activation("relu")(x)
        cnn_output = BatchNormalization()(x)
    
    with ipu.keras.PipelineStage(1):
        conv_shape = K.int_shape(cnn_output)

        x = Flatten()(cnn_output)
        x = Dense(64, activation="relu")(x)
        x = BatchNormalization()(x)
        x = Dense(16, activation="relu")(x)
        x = BatchNormalization()(x)
        x = Dense(4, activation="relu")(x)
        x = BatchNormalization()(x)

        x = Concatenate()([x,mlp_output])
        x = Dense(conv_shape[1]*conv_shape[2]*conv_shape[3], activation="relu")(x)
        x = BatchNormalization()(x)

        x = Reshape((conv_shape[1],conv_shape[2],int(conv_shape[3])))(x)
        x = tf.keras.layers.add([x, cnn_output])
    
    #with ipu.keras.PipelineStage(3):
        x = Conv2D(256, (3,3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        
        x = Conv2DTranspose(256, (3,3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        
        x = UpSampling2D(size=(2,2))(x)
        x = Conv2DTranspose(256, (3,3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        
        x = UpSampling2D(size=(2,2))(x)
        x = Conv2DTranspose(128, (3,3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)

        x = UpSampling2D(size=(2,2))(x)
        x = Conv2DTranspose(64, (3,3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        
        x = Conv2DTranspose(32, (3,3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        
        x = Conv2DTranspose(16, (3,3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        
        x = UpSampling2D(size=(2,2))(x)
        x = Conv2DTranspose(1, (3,3), padding="same", activation="linear")(x)

    model = Model(inputs=[cnn_input, mlp_input], outputs = x)
    
    return model



In [13]:
train_files = get_files(data_path_train)
test_files = get_files(data_path_test)

train_dataset = get_dataset(train_files)
test_dataset = get_dataset(test_files)

train_dataset = train_dataset.batch(batch_size, drop_remainder=True).repeat().prefetch(16)
test_dataset = test_dataset.batch(batch_size, drop_remainder=True).repeat().prefetch(16)

In [14]:
start = time.time()
train_dataset
10275*0.7/8
10275*0.3/8
print('Time elapsed:', time.time() - start)

Time elapsed: 7.748603820800781e-05


In [15]:
start = time.time()
print(time.ctime(start))
with strategy.scope():
    model = full_model((256, 256, 1), 4)
    opt = Adam(learning_rate=learning_rate, decay=learning_rate / n_epochs)
    model.compile(loss="mean_squared_error", optimizer=opt, steps_per_execution=steps_per_execution)
    
    callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoints_path, 
        save_weights_only=True,
        #monitor='val_mse',
        mode='max',
        save_best_only=True
    ),
    tf.keras.callbacks.CSVLogger(
        filename=checkpoints_path + '.csv')
    ]
    
    
    
    model.set_gradient_accumulation_options(
        gradient_accumulation_steps_per_replica=gradient_accumulation_steps_per_replica)
    model.set_pipelining_options(
        gradient_accumulation_steps_per_replica=gradient_accumulation_steps_per_replica)
    model.fit(x=train_dataset, validation_data=test_dataset, epochs=n_epochs, steps_per_epoch=steps_per_epoch,
             validation_steps=validation_steps, callbacks=callbacks)
    #model.save(model_path + '/' + model_name + '.h5')

end = time.time()
print(time.ctime(end))

Fri Oct 29 06:11:27 2021
Epoch 1/100
896/896 [==============================] - 741s 827ms/step - loss: 29.0700 - val_loss: 15.8787
Epoch 2/100
896/896 [==============================] - 83s 92ms/step - loss: 8.9973 - val_loss: 4.3021
Epoch 3/100
896/896 [==============================] - 76s 84ms/step - loss: 3.4564 - val_loss: 3.2807
Epoch 4/100
896/896 [==============================] - 73s 81ms/step - loss: 3.3034 - val_loss: 2.4279
Epoch 5/100
896/896 [==============================] - 69s 77ms/step - loss: 1.7880 - val_loss: 2.6783
Epoch 6/100
896/896 [==============================] - 67s 74ms/step - loss: 2.3366 - val_loss: 2.2089
Epoch 7/100
896/896 [==============================] - 69s 77ms/step - loss: 2.5923 - val_loss: 2.5739
Epoch 8/100
896/896 [==============================] - 70s 78ms/step - loss: 1.6605 - val_loss: 1.8370
Epoch 9/100
896/896 [==============================] - 52s 59ms/step - loss: 1.4407 - val_loss: 2.0844
Epoch 10/100
896/896 [======================

In [16]:
print(start, end)
print("Starting time: ", time.ctime(start))
print("Ending time: ", time.ctime(end))
print("Time elapsed: ", datetime.timedelta(seconds=round(end - start)))
print('')
print("Number of parameters: ", model.count_params())
print("")

1635502287.8490686 1635508155.6447768
Starting time:  Fri Oct 29 06:11:27 2021
Ending time:  Fri Oct 29 07:49:15 2021
Time elapsed:  1:37:48

Number of parameters:  8244633



In [ ]:
logging.info("training model...")
# TODO properly compute steps for progress bar (low priority)
steps_per_epoch = len(train_files) * 10 // batch_size

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoints_path, 
        save_weights_only=True,
        monitor='val_mse',
        mode='max',
        save_best_only=True
    )
]
model.fit(x=train_dataset, validation_data=test_dataset, epochs=n_epochs, callbacks=callbacks)

In [28]:
model.summary()
model.count_params()
#tf.keras.utils.plot_model(model, show_shapes=True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 160         input_2[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 16) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 16) 64          activation[0][0]                 
______________________________________________________________________________________________

8244633

In [13]:
pd.read_hdf(meta_data_path)

,$\zeta$,$\theta_{wave}$,$\eta$,$bathy_i$,bathy_file,bathy,uuid,run_id,bathy_source
0,1.000000,0.000000,1.000000,0,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[-1.2872086, -1.2872086, -1.2872086, -1.48720...",88c98d1f-6307-43b5-a8bd-0cc6b23e105e,a,emodnet
1,1.000000,0.000000,1.000000,462,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[-13.052446, -12.792448, -12.792448, -12.5124...",b38a89ab-f021-42dc-808e-e116dd924a46,a,emodnet
2,1.000000,0.000000,50.000000,0,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[-1.2872086, -1.2872086, -1.2872086, -1.48720...",318908b3-1ed1-4117-8226-6eaeb2846a24,a,emodnet
3,1.000000,0.000000,50.000000,462,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[-13.052446, -12.792448, -12.792448, -12.5124...",dee4ae77-a3af-4e73-9f16-54b7ac589158,a,emodnet
4,1.000000,6.283185,1.000000,0,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[-1.2872086, -1.2872086, -1.2872086, -1.48720...",e7bb6f7b-69db-4515-9dcf-6dc009d4d775,a,emodnet
...,...,...,...,...,...,...,...,...,...
1011,13.282544,3.631269,8.666409,100,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[-21.735966, -21.735966, -21.635967, -21.6359...",65382ebd-1b80-4d75-9287-8bb7f2a41db0,a,emodnet
1012,13.581516,2.358778,29.419263,402,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[6.6918297, 6.6918297, 6.6918297, 6.6918297, ...",0402bcb5-7309-4662-8a18-93f1dccde9b4,a,emodnet
1013,8.416452,5.184157,14.352605,151,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[-1.3258033, -1.3258033, -1.3258033, -1.32580...",065553d1-f60d-4d7e-997e-0b07747682be,a,emodnet
1014,1.511433,5.741217,39.150158,367,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[20.617117, 20.617117, 20.017117, 20.017117, ...",06577167-1089-49f4-aea7-2f736f9398ca,a,emodnet
